In [ ]:
pip install optuna

     |████████████████████████████████| 308 kB 5.3 MB/s 
     |████████████████████████████████| 209 kB 41.7 MB/s 
     |████████████████████████████████| 80 kB 8.0 MB/s 
     |████████████████████████████████| 75 kB 4.1 MB/s 
     |████████████████████████████████| 111 kB 49.6 MB/s 
     |████████████████████████████████| 144 kB 46.4 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=23a0313c8f9efcc9d081e5adcf8aa062cc7f7ed11daf0a5e7d9257d8e94927d5
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
import pickle
import xgboost
import optuna
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('train_data.csv', index_col=0)
weight = []

In [ ]:
df=df[df["Weight"]!='Not Available']

In [ ]:
for index,row in df.iterrows():
    weight.append(row["Weight"].split()[0])

In [ ]:
df["weight"]=weight

In [ ]:
df = df.astype({"weight": int})

In [ ]:
X = df.drop(["Weight","weight","ID"],axis=1).to_numpy()

In [ ]:
y = df["weight"].to_numpy()

In [ ]:
kf = KFold(n_splits=5)

In [ ]:
def define_model(trial):
    lr = trial.suggest_float("lr",1e-5,1e-1,log=True)
    n_estimators = trial.suggest_int("n_estimators",1000,3000)
    max_depth = trial.suggest_int("max_depth",3,10)
    subsample = trial.suggest_float("subsample",0.8,1,log=True)
    colsample_bytree = trial.suggest_float("colsample_bytree",0.3,0.8,log=True)
    gamma = trial.suggest_int("gamma",0,5)
    early_stopping_rounds = trial.suggest_int("early_stopping_rounds",10,100)
    return lr,n_estimators,max_depth,subsample,colsample_bytree,gamma,early_stopping_rounds

In [ ]:
def objective(trial):
    lr,n_estimators,max_depth,subsample,colsample_bytree,gamma,early_stopping_rounds = define_model(trial)
    
    mae = []
    
    for train_index, test_index in kf.split(X):
            
            X_train, X_test, y_train, y_test = X[train_index],X[test_index],y[train_index],y[test_index]

            model_xgboost = xgboost.XGBRegressor(objective ='reg:squarederror',
                  n_estimators = n_estimators,learning_rate = lr,max_depth = max_depth,subsample = subsample,colsample_bytree = colsample_bytree,gamma = gamma, seed = 1)

            eval_set = [(X_test,y_test)]
            model_xgboost.fit(X_train,y_train,early_stopping_rounds = early_stopping_rounds,eval_set = eval_set,verbose = False,eval_metric = "mae")
            y_pred = model_xgboost.predict(X_test)
            mae.append(mean_absolute_error(y_test, y_pred))

    filename = str(lr)+str(n_estimators)+str(max_depth)+str(subsample)+str(colsample_bytree)+str(gamma)+str(early_stopping_rounds)
    filename = filename + ".json"

    model_xgboost.save_model(filename)

    acc = sum(mae)/len(mae)

    return acc

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective,n_trials=50)

print("Best trial:")
trial = study.best_trial

print(" Value: ",trial.value)

print(" Params: ")

for key,value in trial.params.items():
    print("  {}: {}".format(key,value))

[I 2021-11-02 12:43:24,039] A new study created in memory with name: no-name-ad9200fe-cab6-4dac-a7cd-3886278dc742
[I 2021-11-02 12:55:28,880] Trial 0 finished with value: 170.35404799258873 and parameters: {'lr': 4.486365122761064e-05, 'n_estimators': 2311, 'max_depth': 7, 'subsample': 0.9571045003672071, 'colsample_bytree': 0.6949873096950373, 'gamma': 3, 'early_stopping_rounds': 59}. Best is trial 0 with value: 170.35404799258873.
[I 2021-11-02 13:05:57,294] Trial 1 finished with value: 27.414316247957277 and parameters: {'lr': 0.003063497890536736, 'n_estimators': 1747, 'max_depth': 8, 'subsample': 0.9634927963360753, 'colsample_bytree': 0.6915140941626904, 'gamma': 0, 'early_stopping_rounds': 81}. Best is trial 1 with value: 27.414316247957277.
[I 2021-11-02 13:07:13,055] Trial 2 finished with value: 27.387844246802054 and parameters: {'lr': 0.023458942153364148, 'n_estimators': 2761, 'max_depth': 5, 'subsample': 0.9544373049552803, 'colsample_bytree': 0.6298725733546409, 'gamma': 

Best trial:
 Value:  27.376270629318903
 Params: 
  lr: 0.011334813373338989
  n_estimators: 2333
  max_depth: 5
  subsample: 0.8486443999034436
  colsample_bytree: 0.767195468916397
  gamma: 2
  early_stopping_rounds: 55
